<h1 align="center">PINCER manuscript tables</h1>

Pharmacist-led information technology intervention (PINCER) is an effective way to reduce errors in prescription and medication monitoring which are responsible for a large proportion of adverse drug events. This is achieved through the monitoring of 14 PINCER indicators.

The purpose of this notebook is to generate the tables required for PINCER manuscript.

In [47]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as mpllimg
import matplotlib as mpl
import os
import json
from IPython.display import HTML, display, Markdown
mpl.rcParams['figure.dpi']= 200

def show_image(path):
    img=mpllimg.imread(path)
    img_plot = plt.imshow(img)
    plt.axis('off')
    plt.show()

def counts_table_read(folder):
    df = pd.read_csv(f"{folder}/demographics_summary.csv", index_col=False)
    # df.rename(columns={'Unnamed: 0': 'Attribute'}, inplace=True)
    # df.rename(columns={'Unnamed: 1': 'Category'}, inplace=True)
    df.set_index(["demographic", "level"], inplace=True)
    df.sort_index(inplace=True)
    return df
    
def rename_index_categories(df, attribute, replacement_dict):
    index = df.index.values
    for old, new in replacement_dict.items():
        for i, item in enumerate(index):
            if item == (attribute, old):
                index[i] = (attribute, new)
    df.index = pd.MultiIndex.from_tuples(index)
    return df

def reformat_data_tpp(df):
    high_level_ethnicities = {
        "1": "White",
        "2": "Mixed",
        "3": "South Asian",
        "4": "Black",
        "5": "Other",
    }
    imd_categories = {"0": "Missing",
                      "1": "Most deprived 1",
                      "5": "Least deprived 5"}
    region_categories = {"East": "East of England"}
    df = rename_index_categories(df, "ethnicity", high_level_ethnicities)
    df = rename_index_categories(df, "imd", imd_categories)
    df = rename_index_categories(df, "region", region_categories)
    return df


def reformat_data_emis(df):
    high_level_ethnicities = {
        "1": "White",
        "2": "Mixed",
        "3": "South Asian",
        "4": "Black",
        "5": "Other",
    }
    imd_categories = {"0": "Missing",
                      "1": "Most deprived 1",
                      "5": "Least deprived 5" }
    region_categories = {"East": "East of England"}
    df = rename_index_categories(df, "ethnicity", high_level_ethnicities)
    df = rename_index_categories(df, "imd", imd_categories)
    df = rename_index_categories(df, "region", region_categories)
    return df


## Population description

The first table to be generated describes the patient population used in the PINCER analysis, in terms of age group, sex, region, IMD (indices of multiple deprivation) and ethnicity. Counts and percentages are presented, for both TPP and EMIS.

In [49]:
tpp_population = counts_table_read( "../output" )


count  total   perc
demographic level                     
age_band    0-19    1125   9093  12.37
            20-29   1123   9093  12.35
            30-39   1143   9093  12.57
            40-49   1188   9093  13.06
            50-59   1180   9093  12.98

In [51]:

tpp_attribute_counts = tpp_population.reset_index().groupby('demographic').sum()
tpp_total = tpp_attribute_counts.loc["sex","count"]

tpp_population_formatted = reformat_data_tpp(tpp_population)


In [53]:
emis_population = counts_table_read("../output")

emis_attribute_counts = emis_population.reset_index().groupby('demographic').sum()
emis_total = emis_attribute_counts.loc["sex", "count"]

emis_population_formatted = reformat_data_emis(emis_population)


,level_0,level_1,count,total,perc
0,age_band,0-19,1125,9093,12.37
1,age_band,20-29,1123,9093,12.35
2,age_band,30-39,1143,9093,12.57
3,age_band,40-49,1188,9093,13.06
4,age_band,50-59,1180,9093,12.98
5,age_band,60-69,1086,9093,11.94
6,age_band,70-79,1144,9093,12.58
7,age_band,80+,1055,9093,11.60
8,age_band,missing,49,9093,0.54
9,care_home_type,PC,437,9093,4.81


In [54]:

table_1 = pd.merge(left=tpp_population_formatted.reset_index(),
                      right=emis_population_formatted.reset_index(),
                      how="outer",
                      left_on=['level_0', 'level_1'],
                      right_on=['level_0', 'level_1'])

table_1 = table_1.assign(total_count=table_1.count_x + table_1.count_y)
table_1 = table_1.assign(total_perc=round(
    100 * table_1.total_count / (tpp_total + emis_total), 2))

table_1.rename(columns={'level_0': 'demographic',
                           'level_1': 'level',
                           'total_x': 'TPP total',
                           'count_x': 'TPP count',
                           'perc_x': 'TPP percentage',
                           'count_y': 'EMIS count',
                           'total_y': 'EMIS total',
                           'perc_y': 'EMIS percentage'}, inplace=True)


## Cohort description

In [55]:
HTML(table_1.to_html(index=False))


demographic,level,TPP count,total_x,perc_x,EMIS count,total_y,perc_y,total_count,total_perc
age_band,0-19,1125,9093,12.37,1125,9093,12.37,2250,12.37
age_band,20-29,1123,9093,12.35,1123,9093,12.35,2246,12.35
age_band,30-39,1143,9093,12.57,1143,9093,12.57,2286,12.57
age_band,40-49,1188,9093,13.06,1188,9093,13.06,2376,13.06
age_band,50-59,1180,9093,12.98,1180,9093,12.98,2360,12.98
age_band,60-69,1086,9093,11.94,1086,9093,11.94,2172,11.94
age_band,70-79,1144,9093,12.58,1144,9093,12.58,2288,12.58
age_band,80+,1055,9093,11.60,1055,9093,11.60,2110,11.60
age_band,missing,49,9093,0.54,49,9093,0.54,98,0.54
care_home_type,PC,437,9093,4.81,437,9093,4.81,874,4.81


## Indicator counts

The second table to be generated describes the number of patients, the number of events and the number or practices for each indicator. These counts are presented for TPP and EMIS separately, and together.

In [56]:
with open('../output/indicator_summary_statistics.json') as f:
    tpp_summary = json.load(f)["summary"]

tpp_indicator_counts = pd.DataFrame.from_dict(tpp_summary).T.reset_index()


In [57]:

with open('../output/indicator_summary_statistics.json') as f:
    emis_summary = json.load(f)["summary"]

emis_indicator_counts = pd.DataFrame.from_dict(emis_summary).T.reset_index()


        index   events  patients  num_practices  percent_practice
0           a  11500.0    6872.0           41.0             100.0
1           b  11500.0    6870.0           41.0             100.0
2           c  11500.0    6887.0           41.0             100.0
3           d  11500.0    6913.0           41.0             100.0
4           g  11500.0    6985.0           41.0             100.0
5           i  11500.0    6957.0           41.0             100.0
6           k  11500.0    6957.0           41.0             100.0
7          ac  11500.0    6898.0           41.0             100.0
8   me_no_fbc  11500.0    6896.0           41.0             100.0
9   me_no_lft  11500.0    6887.0           41.0             100.0
10         am  11500.0    6912.0           41.0             100.0
11          e   2634.0    2325.0           41.0             100.0
12          f   2724.0    2396.0           41.0             100.0
13         li   1179.0    6880.0           41.0             100.0


In [58]:
table_2 = pd.merge(left=tpp_indicator_counts,
                   right=emis_indicator_counts,
                   how="outer",
                   left_on=['index'],
                   right_on=['index'])

table_2.rename(columns={'index': 'Indicator',
                        'events_x': 'Events (TPP)',
                        'patients_x': 'Patients (TPP)',
                        'num_practices_x': 'Num practices (TPP)',
                        'percent_practice_x': '% practices (TPP)',
                        'events_y': 'Events (EMIS)',
                        'patients_y': 'Patients (EMIS)',
                        'num_practices_y': 'Num practices (EMIS)',
                        'percent_practice_y': '% practices (EMIS)' }, inplace=True)

HTML(table_2.to_html(index=False))


Indicator,Events (TPP),Patients (TPP),Num practices (TPP),% practices (TPP),Events (EMIS),Patients (EMIS),Num practices (EMIS),% practices (EMIS)
a,11500.0,6872.0,41.0,100.0,11500.0,6872.0,41.0,100.0
b,11500.0,6870.0,41.0,100.0,11500.0,6870.0,41.0,100.0
c,11500.0,6887.0,41.0,100.0,11500.0,6887.0,41.0,100.0
d,11500.0,6913.0,41.0,100.0,11500.0,6913.0,41.0,100.0
g,11500.0,6985.0,41.0,100.0,11500.0,6985.0,41.0,100.0
i,11500.0,6957.0,41.0,100.0,11500.0,6957.0,41.0,100.0
k,11500.0,6957.0,41.0,100.0,11500.0,6957.0,41.0,100.0
ac,11500.0,6898.0,41.0,100.0,11500.0,6898.0,41.0,100.0
me_no_fbc,11500.0,6896.0,41.0,100.0,11500.0,6896.0,41.0,100.0
me_no_lft,11500.0,6887.0,41.0,100.0,11500.0,6887.0,41.0,100.0
